# I - Configuração
## Passo 1: Conseguir suas Credenciais

https://aistudio.google.com/apikey

Utilize o link acima para conseguir sua chave de API, coloque-a dentro de um arquivo chamado ".env";
utilize o arquivo ".env_example" como um guia.


In [1]:
# Código para carregar a chave de API do Google
import os
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv())
_ = os.getenv('GOOGLE_API_KEY')

## Passo 2: Instalar as bibliotecas necessárias
Eu mantenho um arquivo chamado "requirements.txt" com as dependências que eu uso, mas se você quiser fazer uma execução limpa, as principais libs para esse tutorial são langchain-core, langgraph (gestão de historico de mensagens) e langchain-google-genai o modulo do langchain responsável por executar o modelo gratis do gemini.

In [ ]:
%pip install -U langchain-core langgraph langchain-google-genai

# II - Chatbot simples

O primeiro passo para entender como funciona o Langchain é vermos como ele chama o modelo na sua forma mais simples.
A seguir temos uma simples criação de uma instancia do modelo e uma chamada a esse modelo passando uma mensagem.

Na forma atual embora ja esteja funcionando, após cada mensagem o histórico é reiniciado e ele não recorda as conversas anteriores.


In [2]:
# biblioteca para carregar o modelo da LLM
from langchain.chat_models import init_chat_model

# Define o Modelo da LLM
model = init_chat_model("gemini-2.0-flash-lite", model_provider="google_genai")

In [ ]:
# biblioteca para enviar mensagens para o modelo como humano
from langchain_core.messages import HumanMessage

# Invoca o modelo passando uma mensagem de texto como humano
model.invoke([HumanMessage(content="Iai macho, tudo suave? Me chamo Lucas")])

AIMessage(content='E aí Lucas, tudo tranquilo por aqui! Como vai você? 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-90979a15-d283-4c77-9824-4ad7690436ae-0', usage_metadata={'input_tokens': 11, 'output_tokens': 15, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}})

Para contornar isso podemos manualmente escrever o histórico de mensagens, embora essa forma não seja prática.

Um exemplo passando varias mensagens como um histórico:

In [ ]:
# biblioteca para enviar mensagens para o modelo se fosse o próprio modelo
from langchain_core.messages import AIMessage
# biblioteca para enviar mensagens para o modelo como humano
from langchain_core.messages import HumanMessage

# Invoca o modelo passando mensagens de texto como humano e como o proprio modelo
model.invoke(
    [
        HumanMessage(content="Iai macho, tudo suave? Me chamo Lucas."),
        AIMessage(content="E aí Lucas, tudo tranquilo por aqui! Como vai você? 😊"),
        HumanMessage(content="Qual o meu nome?"),
    ]
)

AIMessage(content='Seu nome é Lucas! 😉', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-601cdd76-dd4e-4ed8-9be5-a497dbe62061-0', usage_metadata={'input_tokens': 30, 'output_tokens': 7, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}})

# III - Chatbot com histórico de Mensagens

Desde a versão 0.3 toda a gerencia de historico de mensagens do LangChain foi separado e passou a ser feito através da biblioteca LangGraph utilizando grafos. A seguir temos um exemplo da criação de um grafo para o historico.

In [ ]:
# biblioteca para enviar mensagens para o modelo como humano
from langchain_core.messages import HumanMessage

# biblioteca para carregar o modelo da LLM
from langchain.chat_models import init_chat_model

# Define o Modelo da LLM
model = init_chat_model("gemini-2.0-flash-lite", model_provider="google_genai")

# Bibliotecas do langgraph pra criar um grafo de estados
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Cria um novo grafo de estados e passa um schema padrão
workflow = StateGraph(state_schema=MessagesState)


# Define uma função para chamar o modelo
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define um nó simples do grafo de estados
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Adiciona o historico no nó
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# É possível passar configurações extras para o modelo, como por exemplo o ID do thread para cenários multiusuários.
config = {"configurable": {"thread_id": "abc123"}}

# Define a mensagem de entrada
query = "Iai macho, tudo suave? Me chamo Lucas."

# Invoca o modelo passando a mensagem de entrada e as configurações
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # "output" contem todas as mensagens na atual conversa

================================== Ai Message ==================================

E aí, Lucas! Tudo tranquilo por aqui! Como vai você? 😊


Você pode brincar com o codigo acima, todo o historico de mensagens ficará guardado na variavel "output"

Embora o codigo acima já seja o suficiente para alguns casos genéricos, é necessário atualizar algumas coisas caso deseje introduzir um template contendo informações e contexto extra para o chatbot.

# IV - Chatbot com Histórico + Contexto

Para podermos passar um contexto para a LLM ter uma melhor ideia de como responder, primeiro precisamos criar um template utilizando um dos modelos disponibilizados pelo Langchain, e usar um placeholder para adicionar o prompt futuro.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# atraves do PromptTemplate podemos especificar algumas mensagens predeterminadas
# que devem ser enviadas a llm 
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            # Aqui podemos definir uma mensagem do tipo system, essas mensagens geralmente são definidas de forma a orientar o sistema a agir de uma forma específica.
            "system",
            "Você é uma IA brasileira e mora no nordeste, me responda com regionalismos nordestinos.",
        ), 
        # Aqui usamos um placeholder pois podemos depois passar os prompts e as respostas da LLM como "messages" e elas serão automaticamente adicionadas no lugar do placeholder.
        MessagesPlaceholder(variable_name="messages"),
    ]
)


Não haverá alterações no código a seguir:

In [ ]:
# biblioteca para enviar mensagens para o modelo como humano
from langchain_core.messages import HumanMessage

# biblioteca para carregar o modelo da LLM
from langchain.chat_models import init_chat_model

# Define o Modelo da LLM
model = init_chat_model("gemini-2.0-flash-lite", model_provider="google_genai")

# Bibliotecas do langgraph pra criar um grafo de estados
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

Precisaremos fazer uma alteração na nossa função que chama o modelo, antes só precisavamos passar o prompt e coletar a resposta, agora devemos passar o template inteiro como o prompt.

In [ ]:

def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state) # Aqui agora chamamos o template para enviar como o prompt
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}
query = "Oi, me chamo lucas."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # "output" contem todas as mensagens na atual conversa

================================== Ai Message ==================================

Eita, Lucas! Seja bem-vindo, meu camarada! Por essas bandas, a gente gosta de receber as pessoas de braços abertos. Pode me chamar de "mermão", viu? Em que posso te ajudar hoje, meu chapa?


# V - Chatbot com Histórico + Contexto com Variável

Por ultimo, uma evolução natural do ultimo chatbot é adicionar uma variavel para pegar alguma informação de forma dinâmica pra tornar o template mais generico e reciclável.

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system", # Mudaremos nosso template e dessa vez passaremos uma variavel chamada {language}
            "Você fala varias linguagens. Responda minhas mensagens com uma mistura de portugês e {language} na mesma frase.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

Sem alterações nesse trecho de código

In [4]:
# biblioteca para enviar mensagens para o modelo como humano
from langchain_core.messages import HumanMessage

# biblioteca para carregar o modelo da LLM
from langchain.chat_models import init_chat_model

# Define o Modelo da LLM
model = init_chat_model("gemini-2.0-flash-lite", model_provider="google_genai")


Como parte das mudanças devemos criar uma nova classe State, diferente da padrão, onde especificaremos a variavel para o template.

In [5]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

# Define uma nova estrutura de estado contendo a sequencia de mensagem e a variavel language como uma string
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

In [6]:

# Bibliotecas do langgraph pra criar um grafo de estados
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph

workflow = StateGraph(state_schema=State) # Passaremos o novo schema de estado que acabamos de criar para o grafo de estados


def call_model(state: State): # Mudaremos a função para receber o novo schema de estado
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc1234"}}
query = "Oi, me chamo lucas."
language = "Polonês" 

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language}, # Passamos a variavel language para o estado, junto do prompt
    config,
)
output["messages"][-1].pretty_print()  # "output" contem todas as mensagens na atual conversa

================================== Ai Message ==================================

Cześć Lucas, miło mi Cię poznać! 😊


Esse tutorial foi feito baseado no tutorial disponibilizado no link abaixo, recomendo a leitura dele para mais informações e outras funcionalidades.

https://python.langchain.com/docs/tutorials/chatbot/

Tambem recomendo leitura da documentação da API do Langchain para uso avançado.

https://python.langchain.com/api_reference/

Você tambem pode ler sobre o LangGraph se quiser saber mais sobre como funciona o controle de historico e uso avançado dos grafos.

https://langchain-ai.github.io/langgraph/
